In [141]:
from nbformat.v2 import new_metadata

import default.custom_functions as cf
import os
import numpy as np

import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [21]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
os.chdir("/Users/sivaguganjayachandran/PycharmProjects/msu-dse-lab")

json_files = ['json/Middle and High School_Chemistry/' + f for f in os.listdir('json/Middle and High School_Chemistry') if f.endswith('.json')]


In [18]:
vector_dict = cf.load_json_files(json_files)
#vector_dict

In [24]:
query_embeddings = cf.generate_query_embeddings('kjasbcoan c')
query_embeddings.shape

(1, 1536)

In [45]:
vector_dict["metadata"][0]

{'pdf_name': 'Unpacking',
 'document_collection_name': 'Middle and High School Chemistry'}

In [106]:
user_query = 'what is chemistry?'

selected_document = 'ScientificModeling_Mayer'
selected_collection = 'Middle and High School Chemistry'
diversity = 1
top_n = 3

query_params = {
    "query_embeddings" : cf.generate_query_embeddings(user_query),
    "pdf_name": selected_document,
    "document_collection_name": selected_collection,
    "diversity": diversity,
    "top_n": top_n
}

In [ ]:
def query_vector_dict( vector_dict, query_params ):

    ids = vector_dict["ids"]
    documents = vector_dict["documents"]
    metadata = vector_dict["metadata"]
    embeddings = vector_dict["embeddings"]

    # Function to filter metadata or documents based on where conditions
    def apply_filter(data, filter_condition):
        if filter_condition is None:
            return data
        filtered_data = []
        for item in data:
            if all(item.get(key) == value for key, value in filter_condition.items()):
                filtered_data.append(item)
        return filtered_data

    # Apply the `where` and `where_document` filters
    if where:
        metadata = apply_filter(metadata, where)
    if where_document:
        documents = apply_filter(documents, where_document)

    # Ensure we also filter embeddings and ids based on the metadata or documents filter
    # We need to ensure the filtered metadata is indexed correctly
    filtered_metadata = [metadata[i] for i in range(len(metadata)) if metadata[i] in metadata]
    filtered_ids = [ids[i] for i in range(len(ids)) if metadata[i] in filtered_metadata]
    filtered_documents = [documents[i] for i in range(len(documents)) if metadata[i] in filtered_metadata]
    filtered_embeddings = [embeddings[i] for i in range(len(embeddings)) if metadata[i] in filtered_metadata]

    # Calculate the cosine similarity for query_embeddings or query_texts
    if query_embeddings is not None:
        similarities = cosine_similarity(query_embeddings, filtered_embeddings)
    elif query_texts is not None:
        # Generate embeddings for the query_texts
        query_embeddings = generate_embeddings(query_texts)
        similarities = cosine_similarity(query_embeddings, filtered_embeddings)
    else:
        raise ValueError("Either query_embeddings or query_texts must be provided.")

    # Get the closest neighbors (sorted by descending similarity)
    closest_indices = np.argsort(similarities, axis=1)[:, ::-1][:, :n_results]

    # Prepare the results
    results = {
        "ids": [filtered_ids[i] for i in closest_indices.flatten()],
        "documents": [filtered_documents[i] for i in closest_indices.flatten()],
        "metadata": [filtered_metadata[i] for i in closest_indices.flatten()],
        "distances": [similarities[0, i] for i in closest_indices.flatten()]
    }

    # Include only the specified fields
    filtered_results = {}
    if "embeddings" in include:
        filtered_results["embeddings"] = [filtered_embeddings[i] for i in closest_indices.flatten()]
    if "metadatas" in include:
        filtered_results["metadata"] = [filtered_metadata[i] for i in closest_indices.flatten()]
    if "documents" in include:
        filtered_results["documents"] = [filtered_documents[i] for i in closest_indices.flatten()]
    if "distances" in include:
        filtered_results["distances"] = [similarities[0, i] for i in closest_indices.flatten()]

    return filtered_results

In [46]:
def print_dict_outline(d, indent=0):
    for key, value in d.items():
        print("  " * indent + str(key))  # Indent for hierarchy
        if isinstance(value, dict):  # If value is a dict, go deeper
            print_dict_outline(value, indent + 1)

print_dict_outline(vector_dict)

ids
documents
metadata
embeddings


In [77]:
vector_dict.keys()

dict_keys(['ids', 'documents', 'metadata', 'embeddings'])

In [69]:
vector_dict.keys()

dict_keys(['ids', 'documents', 'metadata', 'embeddings'])

In [85]:
new_dict = {
    'metadata' : [],
    'ids' : [],
    'embeddings' : [],
    'documents' : []
}

if query_params['document_collection_name'] is not None or query_params['document_collection_name'] != 'All':

    for i in range(len(vector_dict['ids'])) :

        if vector_dict['metadata'][i]['document_collection_name'] == query_params['document_collection_name'] :
            new_dict['metadata'].append(vector_dict['metadata'][i])
            new_dict['ids'].append(vector_dict['ids'][i])
            new_dict['embeddings'].append(vector_dict['embeddings'][i])
            new_dict['documents'].append(vector_dict['documents'][i])


In [91]:


if query_params['pdf_name'] is not None or query_params['pdf_name'] != 'All':

    for i in range(len(vector_dict['ids'])) :

        if vector_dict['metadata'][i]['pdf_name'] == query_params['pdf_name'] :
            new_dict['metadata'].append(vector_dict['metadata'][i])
            new_dict['ids'].append(vector_dict['ids'][i])
            new_dict['embeddings'].append(vector_dict['embeddings'][i])
            new_dict['documents'].append(vector_dict['documents'][i])

if query_params['pdf_name'] is not None or query_params['pdf_name'] != 'All' and query_params['document_collection_name'] is not None or query_params['document_collection_name'] != 'All':

    for i in range(len(vector_dict['ids'])) :

        if vector_dict['metadata'][i]['pdf_name'] == query_params['pdf_name'] and vector_dict['metadata'][i]['document_collection_name'] == query_params['document_collection_name'] :
            new_dict['metadata'].append(vector_dict['metadata'][i])
            new_dict['ids'].append(vector_dict['ids'][i])
            new_dict['embeddings'].append(vector_dict['embeddings'][i])
            new_dict['documents'].append(vector_dict['documents'][i])

dict_keys(['metadata', 'ids', 'embeddings', 'documents'])

In [94]:
list = []
for i in range(len(new_dict['metadata'])) :
    list.append(new_dict['metadata'][i]['pdf_name'])

list = set(list)
list

{'AssessModel',
 'DeBarger etal Constructing Assessment Item',
 'DesignPrinciples_3DAssessment',
 'DesingScientificExplanationTasks',
 'Evaluating3DTasks',
 'Gotwals_LP_Assessment',
 'Harris et al_Assessment',
 'Harris et al_NSTABook',
 'Hickey_3DLearning',
 'KnowingWhatStudentKnow',
 'NGSS_PhysicalScience',
 'Pellogrino_TrasferableKnowledge',
 'ScientificModeling_Mayer',
 'SystemsforStateScienceAssessment',
 'Unpacking',
 'UnpackingCCC',
 'UnpackingSEP',
 'UnpackingSEP_Examples',
 'UsableKnowledge_Assessment'}

In [107]:
new_dict = {
    'metadata' : [],
    'ids' : [],
    'embeddings' : [],
    'documents' : []
}

pdf_key_1 = query_params['pdf_name'] is not None
collection_key_1 = query_params['document_collection_name'] is not None


for i in range(len(vector_dict['ids'])):
    metadata = vector_dict['metadata'][i]

    pdf_key_2 = not pdf_key_1 or metadata['pdf_name'] == query_params['pdf_name']
    collection_key_2 = not collection_key_1 or metadata['document_collection_name'] == query_params['document_collection_name']

    if pdf_key_2 and collection_key_2 :
        new_dict['metadata'].append(vector_dict['metadata'][i])
        new_dict['ids'].append(vector_dict['ids'][i])
        new_dict['embeddings'].append(vector_dict['embeddings'][i])
        new_dict['documents'].append(vector_dict['documents'][i])


In [99]:
query_params

{'query_embeddings': array([[-0.00547445, -0.03982355, -0.00091198, ..., -0.0407291 ,
         -0.00157571,  0.00651377]]),
 'pdf_name': 'Unpacking',
 'document_collection_name': 'Middle and High School Chemistry',
 'diversity': 1,
 'top_n': 3}

In [108]:
new_dict['ids']

[0, 1, 2, 3, 4]

In [269]:
def query_vector_dict(vector_dict, query_params):

    new_dict = {
        'metadata' : [],
        'ids' : [],
        'embeddings' : [],
        'documents' : []
    }

    pdf_key_1 = query_params['pdf_name'] is not None
    collection_key_1 = query_params['document_collection_name'] is not None

    #filter knowledge base basis user parameters
    for i in range(len(vector_dict['ids'])):
        metadata = vector_dict['metadata'][i]

        pdf_key_2 = not pdf_key_1 or metadata['pdf_name'] == query_params['pdf_name']
        collection_key_2 = not collection_key_1 or metadata['document_collection_name'] == query_params['document_collection_name']

        if pdf_key_2 and collection_key_2 :
            new_dict['metadata'].append(vector_dict['metadata'][i])
            new_dict['ids'].append(vector_dict['ids'][i])
            new_dict['embeddings'].append(vector_dict['embeddings'][i])
            new_dict['documents'].append(vector_dict['documents'][i])


    #calculate similarity score and MMR(maximal marginal relevance) to fetch top_n documents
    l = query_params['diversity'] #lambda

    if query_params['top_n'] <= len(new_dict['ids']):
        top_n = query_params['top_n']
    else:
        top_n = len(new_dict['ids'])

    similarities = cosine_similarity(query_params['query_embeddings'], new_dict['embeddings'])
    mmr = l * similarities - (1 - l) * similarities.max(axis=1, keepdims=True)

    indices = np.argsort(mmr)[:,::-1][:,:top_n]

    final_dict = {
        'metadata' : [],
        'ids' : [],
        #'embeddings' : [],
        'documents' : []
    }

    #create final doct basis relevant and diverse indices
    for i in indices[0]:
        final_dict['metadata'].append(new_dict['metadata'][i])
        final_dict['ids'].append(new_dict['ids'][i])
        #final_dict['embeddings'].append(new_dict['embeddings'][i])
        final_dict['documents'].append(new_dict['documents'][i])

    return final_dict

In [270]:
user_query = 'what is chemistry?'

selected_document = None
selected_collection = 'Middle and High School Chemistry'
diversity = 0.5
top_n = 2

query_params = {
    "query_embeddings" : cf.generate_query_embeddings(user_query),
    "pdf_name": selected_document,
    "document_collection_name": selected_collection,
    "diversity": diversity,
    "top_n": top_n
}

In [271]:
result = query_vector_dict(vector_dict, query_params)

In [272]:
result

{'metadata': [{'pdf_name': 'UsableKnowledge_Assessment',
   'document_collection_name': 'Middle and High School Chemistry'},
  {'pdf_name': 'UsableKnowledge_Assessment',
   'document_collection_name': 'Middle and High School Chemistry'}],
 'ids': [6, 9],
 'documents': ['Usable Assessments7As a first step in our design process, we began with national standards to identify keymiddle school chemistry ideas.  Once we identified the relevant content standards, we unpackedthese relatively succinct statements to clarify the science behind them.  For the first four-weeksegment of the unit, we focused on the concepts of substance, property, and chemical reaction.These concepts are articulated in a national content standard about substances and properties(NRC, 1996, p. 154) and a national content standard about chemical reactions (AAAS, 1990,p.47) (see Table 1).Table 1From National Standard to Learning PerformanceStandardClarifying the StandardLearning PerformanceLP1 – Students identify substanc

In [273]:
vector_dict['metadata'][0]

{'pdf_name': 'Unpacking',
 'document_collection_name': 'Middle and High School Chemistry'}

In [278]:
def get_dict_list(vector_dict):
    result_dict = {}

    for metadata in vector_dict['metadata']:
        collection_name = metadata['document_collection_name']
        pdf_name = metadata['pdf_name']

        if collection_name not in result_dict:
            result_dict[collection_name] = []

        if pdf_name not in result_dict[collection_name]:
            result_dict[collection_name].append(pdf_name)

    return result_dict

In [281]:
doc_list = get_dict_list(vector_dict)

In [288]:
[*doc_list.keys()]

['Middle and High School Chemistry']

In [277]:
b

{'Middle and High School Chemistry'}